https://www.scirp.org/journal/paperinformation.aspx?paperid=67010

Example 1

$F_1(x_1,x_2) = e^{x_1} + x_1 x_2 - 1 = 0$

$F_2(x_1,x_2) = \sin(x_1 x_2) + x_1 + x_2 - 1 = 0$

Solution: $(x_1, x_2) = (0, 1)$

# Import libraries and modules

In [1]:
import math

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras import layers

# Set random seeds for reproducibility
seed_value = 2023
tf.random.set_seed(seed_value)

2023-11-06 17:56:02.241040: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 17:56:02.242912: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 17:56:02.278049: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 17:56:02.278099: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 17:56:02.278130: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

# Functions

In [2]:
def F1 (x1, x2):

    return math.exp(x1) + x1 * x2 - 1

def F2 (x1, x2):

    return math.sin(x1 * x2)  + x1 + x2 - 1


# Create a custom layer using Keras layers
class Layer2(layers.Layer):
    def __init__(self, **kwargs):
        super(Layer2, self).__init__(**kwargs)

    def call(self, inputs):
        x1_val = inputs[0, 0]
        x2_val = inputs[0, 1]
        exp_func = tf.exp(x1_val)
        x1x2 = tf.multiply(x1_val, x2_val)
        sin_func = tf.sin(x1x2)

        layer_act = tf.constant([
            [
                exp_func,
                x1x2,
                0.0,  # Change this to 0.0 to create a 2D tensor
                sin_func,
                x1_val,
                x2_val,
            ]
        ], dtype=tf.float32)

        layer_act = tf.reshape(layer_act, [1, 6])
        return layer_act


# define activation function for layer2
def activation_layer2_ex1(layer):
    x1_val = tf.squeeze(layer[0, 0])
    x2_val = tf.squeeze(layer[0, 1])
    exp_func = tf.exp(x1_val)
    x1x2 = tf.multiply(x1_val, x2_val)
    sin_func = tf.sin(x1x2)

    layer_act = [
        exp_func,
        x1x2,
        0.0,  # Change this to 0.0 to create a 2D tensor
        sin_func,
        x1_val,
        x2_val,
    ]

    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf.float32)
    layer_2 = tf.reshape(layer_2, [1, 6])

    return layer_2


def activation_layer2_ex3(layer):
    
    x1_val = tf.squeeze(layer[0, 0])
    x2_val = tf.squeeze(layer[0, 1])
    exp_func = tf.exp(x2_val)
    x1_3 = tf.pow(x1_val, 3)

    layer_act = [
        x1_val,
        x1_3,
        x2_val,  # Change this to 0.0 to create a 2D tensor
        x1_val,
        x2_val,
        exp_func,
    ]

    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf.float32)
    layer_2 = tf.reshape(layer_2, [1, 6])

    return layer_2



# Create model
def multilayer_perceptron(x, weights, biases):

    # Reshape input if necessary, matching the shape of the first layer's weights
    x = tf.reshape(x, [1, -1])  # Adjust the shape as needed

    layer_1 = tf.add(tf.matmul(x, weights['w12']), biases['b12'])

    # layer_2 = tf.add(tf.matmul(layer_1, tf.transpose(weights['w23'])), biases['b23'])
    layer_2 = activation_layer2_ex1(layer_1)
    # layer_2 = activation_layer2_ex3(layer_1)
    
    # Output fully connected layer
    output = tf.add(tf.matmul(layer_2, weights['w34']), biases['out'])
    
    return output, layer_1


def loss_function(weights, biases):
    
    output, _= multilayer_perceptron(tf.constant(1.0, dtype=tf.float32), weights, biases)

    return tf.reduce_mean(tf.square(output))


# Train step
def train_step(weights, biases, optimizer):

    with tf.GradientTape() as tape:
        
        loss = loss_function(weights, biases)

    trainable_variables = [weights['w12']]  # list containing only 'w12'
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss        


# Constants

In [3]:
training_steps = 5001  #   5000 + 1
display_step = training_steps // 10

learning_rate = 1e-2

# Model

## Create model

In [4]:
# Network Parameters
num_input = 1 # input layer
num_hidden = [2, 6]
num_output = 2 # output layer


# Given by whether the functions in F1 and F2 contains the variables x1 and x2
w23_flags = [[True, True,  False, True, True, False], [False, True, False, True, False, True]]  
w23_flags = tf.constant(w23_flags, dtype=tf.bool)

# Initialize the weights (w23) with zeros
w23 = tf.constant(tf.zeros(num_hidden, dtype=tf.float32))
# Set the weights to 1 where func_flags is True
w23 = tf.where(w23_flags, tf.ones_like(w23), w23)
w23 = tf.transpose(w23)

# Store layers weight & bias
weights = {
    # Variables x1 and x2
    'w12': tf.Variable(tf.random.normal([num_input, num_hidden[0]])),
    # Whether the functions in F1 and F2 contain the variables x1 and x2
    'w23': w23,
    # The coefficients of the functions in F1 and F2
    'w34': tf.constant([[1, 0], [1, 0], [0, 0], [0, 1], [0, 1], [0, 1]], dtype=tf.float32),
    # 'w34': tf.constant([[3, 0], [1, 0], [1, 0], [0, 1], [0, 2], [0, 1]], dtype=tf.float32),
}

biases = {
    'b12': tf.constant([0, 0], dtype=tf.float32),
    'b23': tf.constant(tf.zeros([num_hidden[1]], dtype=tf.float32)),
    'out': tf.constant([[-1, -1]], dtype=tf.float32),
    # 'out': tf.constant([[1, -2]], dtype=tf.float32),
}

# Stochastic gradient descent optimizer.
optimizer = Adam(learning_rate=learning_rate, name='custom_optimizer_name')

In [5]:
weights['w12']

<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[ 0.29742685, -0.41895387]], dtype=float32)>

In [6]:
x_aux = tf.Variable(tf.random.normal([num_input, num_hidden[0]])).numpy()
x_aux

array([[-0.18302704, -0.09167181]], dtype=float32)

## Train model

In [9]:
for i in range(training_steps):
       
    current_loss = train_step(weights, biases, optimizer)
    if i % display_step == 0:
        print(f"epoch {i} => loss: {current_loss:.16e} ")

epoch 0 => loss: 8.0061846971511841e-01 
epoch 500 => loss: 7.4189774750266224e-06 
epoch 1000 => loss: 2.2737367544323206e-13 
epoch 1500 => loss: 1.7763568394002505e-13 
epoch 2000 => loss: 1.5809575870662229e-13 
epoch 2500 => loss: 5.6843418860808015e-14 
epoch 3000 => loss: 3.0198066269804258e-14 
epoch 3500 => loss: 2.8421709430404007e-14 
epoch 4000 => loss: 1.7763568394002505e-15 
epoch 4500 => loss: 3.0198066269804258e-14 
epoch 5000 => loss: 1.7763568394002505e-15 


In [10]:
x1_val, x2_val = weights['w12'][0][0], weights['w12'][0][1]

display(
    (x1_val, x2_val),
    F1(x1_val, x2_val),
    F2(x1_val, x2_val),
)


(<tf.Tensor: shape=(), dtype=float32, numpy=-3.6665828e-08>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99999994>)

<tf.Tensor: shape=(), dtype=float32, numpy=-1.1920929e-07>

<tf.Tensor: shape=(), dtype=float32, numpy=-1.1920929e-07>

In [11]:
F1F2, solution  = multilayer_perceptron(tf.constant(1.0, dtype=tf.float32), weights, biases)

print(f'x1 = {solution[0][0]}, x2 = {solution[0][1]}')
print(f'Residuals: F1 = {F1F2[0][0]}, F2 = {F1F2[0][1]}')

x1 = -3.6665827707338394e-08, x2 = 0.9999999403953552
Residuals: F1 = -1.1920928955078125e-07, F2 = -1.1920928955078125e-07
